# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark

In [2]:
findspark.init('/home/i-sip_iot/spark-3.0.1-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('LRprc').getOrCreate()

In [6]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [7]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [55]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [57]:
from pyspark.ml.feature import StringIndexer

indexed = StringIndexer(inputCol='Cruise_line', outputCol='cruiseline_cat')
df = indexed.fit(df).transform(df)
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruiseline_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|          16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|          16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|           1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|           1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|           1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|        

In [58]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruiseline_cat']

In [9]:
from pyspark.ml.linalg import Vector

In [10]:
from pyspark.ml.feature import VectorAssembler

In [59]:
vec = VectorAssembler(inputCols=['Age',
                                 'Tonnage',
                                 'passengers',
                                 'length',
                                 'cabins',
                                 'passenger_density',
                                 'cruiseline_cat'], outputCol='features')

In [60]:
dff = vec.transform(df)

In [61]:
dff.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruiseline_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|          16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|          16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|           1.0|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|           1.0|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13

In [62]:
dff.select('features').show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [63]:
df_LR = dff.select(['features', 'crew'])

In [64]:
df_LR.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [65]:
from pyspark.ml.regression import LinearRegression

In [66]:
LR_Model = LinearRegression(featuresCol='features', labelCol='crew', predictionCol='prediction')

In [85]:
df_train, df_test = df_LR.randomSplit([0.7,0.3])

In [86]:
df_train.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                99|
|   mean| 7.872424242424252|
| stddev|3.4579267384353156|
|    min|               1.6|
|    max|              21.0|
+-------+------------------+



In [87]:
df_test.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                59|
|   mean|7.6628813559322015|
| stddev|3.6047124572528673|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [88]:
trained_model = LR_Model.fit(df_train)

In [89]:
result = trained_model.evaluate(df_test)

In [90]:
result.r2

0.9556525640992206

In [91]:
result.rootMeanSquaredError

0.7526496702291248

In [92]:
result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.2429014052051564|
| -0.7726207209184874|
| -0.5580377229645244|
| -1.4744348468429749|
| -0.7847657379260919|
| -0.6420860747783532|
| -0.2895879042580365|
|  0.7122855582047656|
| -0.6421987161650637|
|  1.0003056416821305|
|  1.0003056416821305|
| 0.06851736318011525|
|-0.40426449196299075|
| -0.7117460648024885|
| -0.6329831724600599|
|  -1.142619425739321|
|  -0.788492159271831|
|  0.8340224728605854|
|  0.6853698746541816|
|  0.8640865459468383|
+--------------------+
only showing top 20 rows



In [93]:
unlabeled_data = df_LR.select('features')

In [94]:
final_result = trained_model.transform(unlabeled_data)

In [95]:
final_result.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,30.276999999...|4.3953626361127425|
|[6.0,30.276999999...|4.3953626361127425|
|[26.0,47.262,14.8...| 6.307355954751715|
|[11.0,110.0,29.74...|12.246336864117087|
|[17.0,101.353,26....|10.869654176861646|
|[22.0,70.367,20.5...|  8.71002674962089|
|[15.0,70.367,20.5...| 8.741886907734921|
|[23.0,70.367,20.5...| 8.717963881221984|
|[19.0,70.367,20.5...| 8.723681103098333|
|[6.0,110.23899999...|11.393499428654792|
|[10.0,110.0,29.74...| 12.23298317246006|
|[28.0,46.052,14.5...|  6.21293020565875|
|[18.0,70.367,20.5...|  8.72823255425748|
|[17.0,70.367,20.5...| 8.732784005416626|
|[11.0,86.0,21.24,...| 9.573341475047673|
|[8.0,110.0,29.74,...|12.242086074778353|
|[9.0,88.5,21.24,9...| 9.587714441795235|
|[15.0,70.367,20.5...| 8.741886907734921|
|[12.0,88.5,21.24,...|10.431755860763626|
|[20.0,70.367,20.5...| 8.719129651939184|
+--------------------+------------